In [1]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

In [2]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

In [3]:
from langchain_community.graphs import Neo4jGraph

## config vector store

In [4]:
import re
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from docx import Document
from langchain_core.tools import tool
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load the .docx file and extract the text content
config_document = Document("RTB Config document compilation.docx")

config_text = ""
for paragraph in config_document.paragraphs:
    config_text += paragraph.text + "\n"

# Split the config content into individual documents based on headings (assuming "##" as a delimiter)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300)
config_doc_chunks = text_splitter.split_text(config_text)

# Initialize OpenAI embeddings for ChromaDB
OpenAIembedding = OpenAIEmbeddings(model="text-embedding-3-large")  # Updated model name

# Initialize ChromaDB vector store from text and embeddings
vector_store = Chroma.from_texts(config_doc_chunks, OpenAIembedding, collection_name="config_document")


# VectorStoreRetriever class using ChromaDB
class VectorStoreRetriever:
    def __init__(self, vector_store: Chroma):
        self.vector_store = vector_store

    def query(self, query: str, k: int = 5) -> list[dict]:
        # Perform a similarity search in ChromaDB
        results = self.vector_store.similarity_search_with_score(query, k=k)
        return [
            {
                "page_content": result[0].page_content,  
                "similarity_score": result[1],
                
            }
            for result in results
        ]

# Initialize retriever with the vector store
retriever = VectorStoreRetriever(vector_store)

@tool
def lookup_config(query: str) -> str:
    """Look up relevant parts of the config document based on the query."""
    docs = retriever.query(query, k=2)
    return "\n\n".join([doc["page_content"] for doc in docs])

query = "how to control a date?"
result = lookup_config(query)
result


C:\Users\TusharJain\AppData\Local\Temp\ipykernel_30536\3292921193.py:20: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  OpenAIembedding = OpenAIEmbeddings(model="text-embedding-3-large")  # Updated model name
C:\Users\TusharJain\AppData\Local\Temp\ipykernel_30536\3292921193.py:53: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = lookup_config(query)


'End of Week - the date returned by the Method will be adjusted to the Sunday on or after.\nEnd of Month - the date returned by the Method will be adjusted to the last day of the calendar month.\nEnd of Quarter - the date returned by the Method will be adjusted to the last day of the calendar quarter.\nEnd of Year - the date returned by the Method will be adjusted to the last day of the calendar year.\nBaseline dates are further adjusted, as required, by settings in the Define Date Rules activity.\nDefine Date Rules\nIn this activity you define rules that determine the dates for Billing Items. Date Rules are used to determine Service Period Start, Service Period End, and Billing Date.\nTo define a new date rule, click the New Entries button. Provide a unique identifier and a descriptive name. There is no reserved namespace for the two-character identifier, so they do not need to start with X, Y, or Z.\nMake the following settings:\nBaseline Date - select a baseline date from the values

In [5]:
from docx import Document
document = Document("Calss_export_word.docx")
document

text = ""
for paragraph in document.paragraphs:
    text += paragraph.text + "\n"

cleaned_text = text.replace('\xa0', ' ')

In [6]:
method_pattern = r"(?<!\w)METHOD\s+([\w_]+)\.\s*(.*?)\s*ENDMETHOD\."

# Extract methods and contents
methods_dict = {name.lower().strip(): content.strip() for name, content in re.findall(method_pattern, cleaned_text, re.DOTALL)}

In [7]:
def extract_method_details(text):
    method_pattern = r"class-methods\s+([^\s]+)\s+"
    importing_pattern = r"importing\s+(.*?)\s+(?:returning|exporting)"
    exporting_pattern = r"exporting\s+(.*?)\s*\."
    returning_pattern = r"returning\s+(.*?)\s*\."
    

    method_details = {}
    for match in re.finditer(method_pattern, text):
        method_name = match.group(1).lower()
        method_details[method_name] = {}

        importing_match = re.search(importing_pattern, text[match.end():])
        if importing_match:
            method_details[method_name]['importing'] = [param.strip() for param in importing_match.group(1).split('\n') if param.strip()]

        returning_match = re.search(returning_pattern, text[match.end():], re.DOTALL)
        if returning_match:
            method_details[method_name]['exporting'] = [param.strip() for param in returning_match.group(1).split('\n') if param.strip()]

        exporting_match = re.search(exporting_pattern, text[match.end():], re.DOTALL)
        if exporting_match:
            method_details[method_name]['exporting'] = [param.strip() for param in exporting_match.group(1).split('\n') if param.strip()]

    return method_details

method_params = extract_method_details(cleaned_text)
print(method_params)

{'process_realtime_data': {'importing': ['!I_KSCHL type KSCHL'], 'exporting': ['!ET_RETURN type /CGDC/T_MESSAGE']}, 'get_previously_billed': {'importing': ['!I_KSCHL type KSCHL'], 'exporting': ['!ET_PREV_BILL_DETAILS type /CGDC/T_KONV_DET', '!ET_PREV_BILL_SUMMARY type /CGDC/T_VBRP_DET']}, 'subsequent_doc_data_update': {'importing': ['!I_KSCHL type KSCHL'], 'exporting': ['!ET_KOMV type KOMV_ITAB', '!ET_KOMV_ADD_DATA type /CGDC/TT_CLRQ_CS_CON_DATA']}, 'buffer_costsheet_conditions': {'importing': ['!I_KSCHL type KSCHL'], 'exporting': ['!ET_KOMV type KOMV_ITAB', '!ET_KOMV_ADD_DATA type /CGDC/TT_CLRQ_CS_CON_DATA']}, 'get_costsheet_conditions': {'importing': ['!I_KSCHL type KSCHL'], 'exporting': ['!ET_KOMV type KOMV_ITAB', '!ET_KOMV_ADD_DATA type /CGDC/TT_CLRQ_CS_CON_DATA']}, 'check_costing_sheet_intg': {'importing': ['!I_KSCHL type KSCHL'], 'exporting': ['!E_STATUS type /CGDC/CLRQ_CSIST']}, 'set_clrq_costsheet_conditions': {'importing': ['!I_KSCHL type KSCHL'], 'exporting': ['!ES_KOMV type 

# creating dictionary of the method and class

In [8]:
consolidated_dict = {}

for method_name, method_content in methods_dict.items():
    consolidated_dict[method_name] = {
        "method_code": method_content,
        "importing_parameters": method_params.get(method_name, {}).get('importing', []),
        "exporting_parameters": method_params.get(method_name, {}).get('exporting', []),
        "returning_parameters": method_params.get(method_name, {}).get('returning', []),
    }

print(consolidated_dict)

{'buffer_costsheet_conditions': {'method_code': '**********************************************************\n* Cognitus Technologies Confidential\n**********************************************************\n*\n*  [2019] - [2025] Cognitus Technologies LLC\n*  All Rights Reserved.\n*\n* NOTICE:  All information contained herein is, and remains\n* the property of Cognitus Technologies LLC and its suppliers,\n* if any. The intellectual and technical concepts contained\n* herein are proprietary to Cognitus Technologies LLC and its\n* suppliers and may be covered by U.S. and Foreign Patents,\n* patents in process, and are protected by trade secret or\n* copyright law. Dissemination of this information or\n* reproduction of this material is strictly forbidden unless\n* prior written permission is obtained from Cognitus\n* Consulting LLC.\n**********************************************************\n* VERSION CONTROL (Most recent on top):\n*\n* RELEASE: SAPK-231COINCGDC (CIS-AD S/4 Hana 2023)\n

In [9]:
class_pattern = r"class\s+([^\s]+)\s+"
class_name_match = re.search(class_pattern, cleaned_text)
class_name = class_name_match.group(1).lower() if class_name_match else "unknown_class"
class_details = {class_name: consolidated_dict}

In [10]:
print(class_details.keys())
method_name_to_search = "get_costsheet_conditions"
if method_name_to_search in class_details[class_name]:
    print(f"Method '{method_name_to_search}' found in class '{class_name}'.")
else:
    print(f"Method '{method_name_to_search}' not found in class '{class_name}'.")

dict_keys(['/cgdc/cl_clrq_process'])
Method 'get_costsheet_conditions' found in class '/cgdc/cl_clrq_process'.


In [11]:
from langchain.tools import tool

@tool("parse_class_documentation", return_direct=True)
def parse_class_dict(method_name: str):
    """
    Parses the class documentation to extract the method details for a specific class and method.
    
    Arguments:
    - class_name: The class name for which the method belongs.
    - method_name: The specific method name for which the documentation is requested.

    Returns:
    - The method code and parameters as part of the documentation.
    """
    method_name = method_name.lower()
    if class_name in class_details and method_name in class_details[class_name]:
        method_code = class_details[class_name][method_name]['method_code']
        return method_code
    return "Method not found"


    


In [12]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain.chains.graph_qa.cypher_utils import CypherQueryCorrector, Schema

In [13]:
NEO4J_URI = "neo4j+s://909a82f6.databases.neo4j.io"  # or neo4j+s://xxxx.databases.neo4j.io
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "ERHMOSLUToDtV33RdV3oRpne18Aoie82tOZVqAHl6KE"  # your password
NEO4J_DATABASE = "neo4j"

# Create a Neo4j driver
graph = Neo4jGraph(url=NEO4J_URI , username=NEO4J_USERNAME , password=NEO4J_PASSWORD, enhanced_schema=True)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: 'MATCH (n:`__Covariate__`)\nWITH collect(distinct substring(toString(n.`id`), 0, 50)) AS `id_values`,\n     co

In [14]:
type(graph)

langchain_community.graphs.neo4j_graph.Neo4jGraph

In [15]:
schema = [
    Schema(el["start"], el["type"], el["end"])
    for el in graph.structured_schema.get("relationships")
]

In [16]:
schema

[Schema(left_node='__Chunk__', relation='PART_OF', right_node='__Document__'),
 Schema(left_node='__Chunk__', relation='HAS_COVARIATE', right_node='__Covariate__'),
 Schema(left_node='__Chunk__', relation='HAS_ENTITY', right_node='Type'),
 Schema(left_node='__Chunk__', relation='HAS_ENTITY', right_node='Constants'),
 Schema(left_node='__Chunk__', relation='HAS_ENTITY', right_node='Variable'),
 Schema(left_node='__Chunk__', relation='HAS_ENTITY', right_node='Method'),
 Schema(left_node='__Chunk__', relation='HAS_ENTITY', right_node='Class'),
 Schema(left_node='__Chunk__', relation='HAS_ENTITY', right_node='Table'),
 Schema(left_node='__Chunk__', relation='HAS_ENTITY', right_node='Function'),
 Schema(left_node='__Chunk__', relation='HAS_ENTITY', right_node='Field'),
 Schema(left_node='__Chunk__', relation='HAS_ENTITY', right_node='Parameter'),
 Schema(left_node='__Community__', relation='HAS_FINDING', right_node='Finding'),
 Schema(left_node='Class', relation='RELATED', right_node='Metho

In [71]:
from langchain.tools import tool
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains import GraphCypherQAChain

# Define the Cypher generation template
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
You are an expert in NEO4J and generating CYPHER queries to query a graph database.
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.

Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
If you do not find the answer try the in_community relationship that can be helpful in some cases.
Examples: Here are a few examples of generated Cypher statements for particular questions:

#How many methods exist in the class?
MATCH path = (entity:Method)-[:RELATED]-(class:Class)
RETURN count(path)

#Methods with most dependencies?
MATCH (method:Method)-[r]->(__Entity__)
WHERE entity:Class OR entity:Variable OR entity:Table or entity:Method or entity:Function
WITH method, count(r) AS dependenciesCount
RETURN method.name AS MethodName, dependenciesCount
ORDER BY dependenciesCount DESC

#What are the functions present in a method?
MATCH (method:Method)-[:RELATED]-(function:Function)
RETURN method.name AS MethodName, function.name AS FunctionName

MATCH (method:Method)-[:RELATED]-(function:Function)
WHERE toLower(method.name) CONTAINS toLower("copy_clgrp_cond")
RETURN function.name AS FunctionName
full context: [{{'FunctionName': '/CGDC/CLRQ_DATA_GET'}}]
the full context reurned from the query contains the answer to the question
answer = '/CGDC/CLRQ_DATA_GET'

#what are the functions used in a Get_Buffer_conditions method?
MATCH (method:Method)-[:RELATED]-(function:Function)
WHERE toLower(method.name) = toLower("get_buffer_conditions")
RETURN function.name AS FunctionName

#Find the entities related to? (REALY IMPORTANT QUERY. USE THIS TO GET THE ENTITIES RELATED TO THE METHOD)
MATCH (method:Method)-[:RELATED]-(entity:__Entity__)
WHERE to Lower(method.name) CONTAINS to Lower("get_buffer_conditions")
RETURN entity.name



The question is:
{question}"""

# Create the Cypher generation prompt using LangChain's PromptTemplate
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["question"],
    template=CYPHER_GENERATION_TEMPLATE
)

# Define the tool for generating and executing Cypher queries
@tool("generate_and_run_cypher_query", return_direct=True)
def generate_and_run_cypher_query(question: str):
    
    """
    Tool to generate a Cypher query based on a schema and a question,
    and then execute the query on the provided graph.
    """
    print("Starting 'generate_and_run_cypher_query'...")
    NEO4J_URI = "neo4j+s://909a82f6.databases.neo4j.io"  # or neo4j+s://xxxx.databases.neo4j.io
    NEO4J_USERNAME = "neo4j"
    NEO4J_PASSWORD = "ERHMOSLUToDtV33RdV3oRpne18Aoie82tOZVqAHl6KE"  # your password
    
  
# Create a Neo4j driver
    graph = Neo4jGraph(url=NEO4J_URI , username=NEO4J_USERNAME , password=NEO4J_PASSWORD, enhanced_schema=True)
    
    # Initialize the GraphCypherQAChain with your LLM and graph
    chain = GraphCypherQAChain.from_llm(
        llm=ChatOpenAI(model = 'gpt-4', temperature=0, max_tokens = 4096),  # Your LLM model (e.g., OpenAI)
        graph=graph,  # Your Neo4j connection or graph structure
        cypher_prompt=CYPHER_GENERATION_PROMPT,
        allow_dangerous_requests=True,
        validate_cypher=True,
        return_intermediate_steps = True,
        return_direct=True,
        top_k=32
    )
    
    def question_modifier(llm, question: str, context) -> str:
                    """Modify the user query based on the method names identified in the user query."""
                    prompt = f"Extract the SAP method name from the user query: {question}. You can refer to the method names in {context}.Just return the method name."
                    llm_response = llm.invoke(prompt)
                    method_name = llm_response.content.strip()
                    return  f"Find the entities related to {method_name}."

    llm = ChatOpenAI(model = 'gpt-4', temperature=0)
    cypher_result = chain.invoke({"schema":schema,"query": question_modifier(llm=llm, question=question, context=method_params.keys()).strip()})

    if type(cypher_result) == list:
        cypher_result = cypher_result[0]
    
    
    return cypher_result


In [72]:
question = "What entities are related to COPY_CLGRP_COND method?"

# Generate and run the Cypher query using the dynamic schema
result = generate_and_run_cypher_query({ "question": question, "context": method_params})
print(result)

Starting 'generate_and_run_cypher_query'...


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: 'MATCH (n:`__Covariate__`)\nWITH collect(distinct substring(toString(n.`id`), 0, 50)) AS `id_values`,\n     co

{'schema': {'node_props': {'__Chunk__': [{'property': 'id', 'type': 'STRING', 'values': ['77afae11468def2f00d30a57c55c02c8', '29a7062d6a5b159a1c53444e91625410', 'f27e159af19bb0bd2bc4755db8c4eadd', '79c03eff500b74c2ad267d55a29bada4', '5282cb0d1d69ad46bab2cff472e40e44', '5b3f236a739ebd41116e0884dfa63c56', 'c2fc0392e43a2af0840b1d661606658d', '5ebc480e6d102e5881c8687a63d51e15', 'a160486478a11ccf6353de7fd3dc2352', '0ab8f2a01b4489561e953aba8045d88a'], 'distinct_count': 52}, {'property': 'n_tokens', 'type': 'INTEGER', 'min': '462', 'max': '1200', 'distinct_count': 2}, {'property': 'text', 'type': 'STRING', 'values': ['class /CGDC/CL_CLRQ_PROCESS definition\n  public\n  ', 'S\n    importing\n      !IT_KOMV type KOMV_ITAB\n    ', 'CGDC/CLRQT optional\n      !IV_VKORG type VKORG opt', 'Q_TRGR .\nprotected section.\n\n  class-data GS_TKA01', '          lwa_calc_group_fun_src   TYPE /cgdc/s_rt', "DC/_CLRQIC'\n        IMPORTING\n          et_dfies  ", "'.\n            CONCATENATE lv_date_where ` AN

In [31]:
import re
from langchain_core.tools import tool

# Function parsing logic
def extract_function_details(text):
    # Adjusted regex pattern to match the function name
    function_name_pattern = r"(?<=FUNCTION\s)(\/\w+\/\w+)"
    # Adjusted regex pattern to match the entire function block
    function_pattern = r"FUNCTION\s+([\w_\/]+)\.\s*(.*?)\s*ENDFUNCTION\."

    # Dictionary to store function name as key and details as value
    function_details = {}

    # Find all function blocks
    for match in re.finditer(function_pattern, text, re.DOTALL):
        # Extract function name using the previously defined function_name_pattern
        function_name_match = re.search(function_name_pattern, match.group(0))
        print(function_name_match)
        if function_name_match:
            function_name = function_name_match.group(1).lower()  # Function name as the key
            function_content = match.group(2).strip()  # Everything between FUNCTION and ENDFUNCTION
            # Store content as the value
            print(function_name)
            print(function_content)
            function_details[function_name] = function_content

    return function_details

# Wrapping into a LangChain tool
@tool("parse_function_documentation", return_direct=True)
def parse_function_documentation(function_name: str):
    """
    Parses ABAP function documentation from the provided text.
    Extracts function names and their respective content between FUNCTION and ENDFUNCTION.
    Helpful in providing details of functions that can be used for understanding a method or class.
    """
    function_name = function_name.lower()
    for function_name in function_params.keys():
        if function_name in function_params:
            return function_params[function_name]

    return extract_function_details(function_text)

# Example usage of the tool

with open('Function clrq_data_get..txt', 'r') as file:
    function_text = file.read()

function_params = extract_function_details(function_text)
function_params


<re.Match object; span=(9, 28), match='/cgdc/clrq_data_get'>
/cgdc/clrq_data_get
*"----------------------------------------------------------------------
*"*"Local Interface:
*"  EXPORTING
*"     REFERENCE(ES_CLRHD) TYPE  /CGDC/S_CLRQHD
*"     REFERENCE(ES_CLRIT) TYPE  /CGDC/S_CLRQIT
*"     REFERENCE(ES_CRGRR) TYPE  /CGDC/S_CLRQGRR
*"     REFERENCE(ET_CRGRRF) TYPE  /CGDC/TT_CLRQGRRF
*"     REFERENCE(ET_CLGRP_ITM_BUFFER) TYPE  /CGDC/TT_CLRQIT
*"     REFERENCE(ET_CLGRP_CND_BUFFER) TYPE  /CGDC/TT_CLRQCN
*"     REFERENCE(ET_PBD_CONDS) TYPE  BAPIEBDRREQUESTCOND_T
*"     REFERENCE(ET_CLRQIT_GRP_CONDS) TYPE  ANY
*"     REFERENCE(ET_CRGRR_KOMV) TYPE  KOMV_TAB
*"     REFERENCE(ET_CLGRP_ITM) TYPE  /CGDC/TT_CLRQIT
*"     REFERENCE(ET_CLGRP_CND) TYPE  /CGDC/TT_CLRQCN
*"     REFERENCE(EV_CONSTR_DT) TYPE  SY-DATUM
*"     REFERENCE(ET_CLRBI) TYPE  /CGDC/TT_CLRQBI_FCAT
*"----------------------------------------------------------------------
**********************************************************
* 

{'/cgdc/clrq_data_get': '*"----------------------------------------------------------------------\n*"*"Local Interface:\n*"  EXPORTING\n*"     REFERENCE(ES_CLRHD) TYPE  /CGDC/S_CLRQHD\n*"     REFERENCE(ES_CLRIT) TYPE  /CGDC/S_CLRQIT\n*"     REFERENCE(ES_CRGRR) TYPE  /CGDC/S_CLRQGRR\n*"     REFERENCE(ET_CRGRRF) TYPE  /CGDC/TT_CLRQGRRF\n*"     REFERENCE(ET_CLGRP_ITM_BUFFER) TYPE  /CGDC/TT_CLRQIT\n*"     REFERENCE(ET_CLGRP_CND_BUFFER) TYPE  /CGDC/TT_CLRQCN\n*"     REFERENCE(ET_PBD_CONDS) TYPE  BAPIEBDRREQUESTCOND_T\n*"     REFERENCE(ET_CLRQIT_GRP_CONDS) TYPE  ANY\n*"     REFERENCE(ET_CRGRR_KOMV) TYPE  KOMV_TAB\n*"     REFERENCE(ET_CLGRP_ITM) TYPE  /CGDC/TT_CLRQIT\n*"     REFERENCE(ET_CLGRP_CND) TYPE  /CGDC/TT_CLRQCN\n*"     REFERENCE(EV_CONSTR_DT) TYPE  SY-DATUM\n*"     REFERENCE(ET_CLRBI) TYPE  /CGDC/TT_CLRQBI_FCAT\n*"----------------------------------------------------------------------\n**********************************************************\n* Cognitus Technologies Confidential\n**

In [20]:
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda

from langgraph.prebuilt import ToolNode


def handle_tool_error(state) -> dict:
    error = state.get("error")
    tool_calls = state["messages"][-1].tool_calls
    return {
        "messages": [
            ToolMessage(
                content=f"Error: {repr(error)}\n please fix your mistakes.",
                tool_call_id=tc["id"],
            )
            for tc in tool_calls
        ]
    }


def create_tool_node_with_fallback(tools: list) -> dict:
    return ToolNode(tools).with_fallbacks(
        [RunnableLambda(handle_tool_error)], exception_key="error"
    )


def _print_event(event: dict, _printed: set, max_length=1500):
    current_state = event.get("dialog_state")
    if current_state:
        print("Currently in: ", current_state[-1])
    message = event.get("messages")
    if message:
        if isinstance(message, list):
            message = message[-1]
        if message.id not in _printed:
            msg_repr = message.pretty_repr(html=True)
            if len(msg_repr) > max_length:
                msg_repr = msg_repr[:max_length] + " ... (truncated)"
            print(msg_repr)
            _printed.add(message.id)

In [32]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph.message import AnyMessage, add_messages


class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [33]:
from langchain_core.messages import HumanMessage


In [34]:
def set_method_params():
    global method_params
    method_params = extract_method_details(cleaned_text)

In [55]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnableConfig
from datetime import datetime

# Define the Assistant class
class Assistant:
    def __init__(self, runnable: Runnable, graph=None, method_params=None):
        #graph = Neo4jGraph(url=NEO4J_URI , username=NEO4J_USERNAME , password=NEO4J_PASSWORD, enhanced_schema=True)
        self.runnable = runnable
        #self.graph = graph
        self.method_params = method_params

    def __call__(self, state: dict, config: RunnableConfig, method_params = method_params):
        while True:
            print(f"Configuration received: {config}")
            messages = state.get("messages", [])
            print(messages)

            if "generate_and_run_cypher_query" in self.runnable.get_name():
                print("Graph Query Tool detected.")
                # method_params = self.method_params if self.method_params else {}
                graph = self.graph if self.graph else None  # Ensure graph is set in the state

                # Prepare the input for the tool
                tool_input = {
                                "question": query,  # Use the extracted query
                                "graph": graph,     # Ensure graph is properly passed
                          # Ensure schema is properly passed
                                "context": method_params.keys()  # Pass method params to the tool
        }
                 # Invoke the runnable with the prepared input
                result = self.runnable.invoke(tool_input)
                print("Result from Graph Query Tool:", result)
            else:
                print("Invoking other tools or processing states.")
                result = self.runnable.invoke(state)


            # Check if LLM returned a valid response or if we need to retry
            if not result.tool_calls and (
                not result.content
                or isinstance(result.content, list)
                and not result.content[0].get("text")
            ):
                messages = state["messages"] + [("user", "Please provide a valid response.")]
                state = {**state, "messages": messages}
            else:
                break
        return {"messages": result}

# Initialize OpenAI's GPT-4 model
llm = ChatOpenAI(model="gpt-4", temperature=0.2, max_tokens=2048)

# Define a prompt template for the assistant to generate proper queries based on user input
primary_assistant_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant focused on understanding SAP ABAP code, "
            "documenting methods, functions, and resolving queries using the provided tools. "
            "For every query, use the tools effectively. When searching in the graph or config, "
            "ensure you understand the context of the class/method/functions."
            "Try and identify the method name from the user query."
            "If the user is asking to print a documentation, use the generate and run query tool first, to check if functions exist or not."
            "Use the output from the parse class documentation to explain what the method or class is doing."
            """When you identify a method is being used, use the function documentation tool to understand the function and create a function documentation. Use the function documentation 
            to enhance the understanding of the method. and then generate a highly technical documentation by incorporating method and function understanding.""" 
            "\n\nUser Query:\n<User>\n{{user_query}}\n</User>"
            "\nCurrent time: {time}.",
        ),
        ("placeholder", "{messages}"),
    ]
).partial(time=datetime.now())

# List of tools that will be bound to the assistant (adjusted for your use case)
part_1_tools = [
    # generate_and_run_cypher_query,  # Tool for graph querying
    lookup_config,  # Tool for retrieving information from the configuration document
    parse_class_dict,  # Tool for extracting class and method information from the doc
    parse_function_documentation,  # Tool to parse and understand functions
]

# Bind the tools to the assistant prompt using LangChain’s tool binding mechanism
part_1_assistant_runnable = primary_assistant_prompt | llm.bind_tools(part_1_tools)

# Example of the Assistant node setup using LangGraph (simplified)
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver

builder = StateGraph(State)

# Define nodes: assistant and tools
builder.add_node("assistant", Assistant(part_1_assistant_runnable))
builder.add_node("tools", create_tool_node_with_fallback(part_1_tools))

# Define edges for control flow
builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools", "assistant")
builder.add_edge("assistant", END)

# Create the memory to save the graph state
memory = MemorySaver()
part_1_graph = builder.compile(checkpointer=memory)


In [25]:
# from langchain_openai import ChatOpenAI
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.runnables import Runnable, RunnableConfig
# from datetime import datetime

# # Define the Assistant class
# class Assistant:
#     def __init__(self, runnable: Runnable, neo4j_uri=None, neo4j_username=None, neo4j_password=None, method_params=None):
#         # Initialize Neo4j graph
#         self.graph = Neo4jGraph(url=neo4j_uri, username=neo4j_username, password=neo4j_password, enhanced_schema=True)
#         self.runnable = runnable
#         self.method_params = method_params

#     def __call__(self, state: dict, config: RunnableConfig,):
#         print(f"Configuration received: {config}")
#         messages = state.get("messages", [])
#         print(messages)

#         # Extract the query from the state (assuming the user query is in state)
#         query = state.get("query", "default query")  # Ensure there's a query

#         # If the tool being invoked is the Graph Query Tool
#         if "generate_and_run_cypher_query" in self.runnable.get_name():
#             print("Graph Query Tool detected.")
#             method_params = self.method_params if self.method_params else {}
#             graph = self.graph if self.graph else None

#             # Check for graph and schema
#             if not graph or not schema:
#                 raise ValueError("Graph or Schema is not properly initialized!")

#             # Prepare input for the tool
#             tool_input = {
#                 "question": query,  # Pass the extracted query
#                 "graph": graph,
#                 "method_params": self.method_params
#             }

#             # Invoke the runnable with the prepared input
#             result = self.runnable.invoke(tool_input)
#             print("Result from Graph Query Tool:", result)
#         else:
#             print("Invoking other tools or processing states.")
#             result = self.runnable.invoke(state)

#         # Retry logic if LLM didn't return a valid response
#         retry_count = 0
#         while retry_count < 3:  # Retry up to 3 times to avoid infinite loop
#             if not result.tool_calls and (
#                 not result.content or (isinstance(result.content, list) and not result.content[0].get("text"))
#             ):
#                 messages = state.get("messages", []) + [("user", "Please provide a valid response.")]
#                 state = {**state, "messages": messages}
#                 result = self.runnable.invoke(state)  # Retry invoking
#                 retry_count += 1
#             else:
#                 break

#         return {"messages": result}

# # Initialize OpenAI's GPT-4 model
# llm = ChatOpenAI(model="gpt-4", temperature=0.2)

# # Define a prompt template for the assistant
# primary_assistant_prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             "You are a helpful assistant focused on understanding SAP ABAP code, "
#             "documenting methods, functions, and resolving queries using the provided tools. "
#             "For every query, use the tools effectively. When searching in the graph or config, "
#             "ensure you understand the context of the class/method/functions."
#             "If the user is asking to print a documentation, use the generate and run query tool first, to check if functions exist or not."
#             """When you identify a method, use the function documentation tool to understand the function and create a function documentation. 
#             Use the function documentation to enhance the understanding of the method, and generate technical documentation.""",
#         ),
#         ("placeholder", "{messages}"),
#     ]
# ).partial(time=datetime.now())

# # List of tools to bind to the assistant
# part_1_tools = [
#     generate_and_run_cypher_query,  # Tool for graph querying
#     lookup_config,  # Tool for retrieving information from the configuration document
#     parse_class_dict,  # Tool for extracting class and method information from the doc
#     parse_function_documentation,  # Tool to parse and understand functions
# ]

# # Bind tools to the assistant prompt using LangChain’s tool binding mechanism
# part_1_assistant_runnable = primary_assistant_prompt | llm.bind_tools(part_1_tools)

# # Example of the Assistant node setup using LangGraph (simplified)
# from langgraph.graph import END, StateGraph, START
# from langgraph.prebuilt import tools_condition
# from langgraph.checkpoint.memory import MemorySaver

# builder = StateGraph(State)

# # Define nodes: assistant and tools
# builder.add_node("assistant", Assistant(part_1_assistant_runnable, neo4j_uri=NEO4J_URI, neo4j_username=NEO4J_USERNAME, neo4j_password=NEO4J_PASSWORD))
# builder.add_node("tools", create_tool_node_with_fallback(part_1_tools))

# # Define edges for control flow
# builder.add_edge(START, "assistant")
# builder.add_conditional_edges("assistant", tools_condition)
# builder.add_edge("tools", "assistant")
# builder.add_edge("assistant", END)

# # Create memory to save the graph state
# memory = MemorySaver()
# part_1_graph = builder.compile(checkpointer=memory)


In [56]:
import shutil
import uuid

# Let's create an example conversation a user might have with the assistant
tutorial_questions = [
    "Write a documentation for the method COPY_CLGRP_COND",
]

# Let's use a UUID for the thread ID for consistent state checkpoints
thread_id = str(uuid.uuid4())

# Configuration, can include things like default class or method names to query
config = {
    "configurable": {
        # The thread_id can be used for persistent checkpoints
        "thread_id": thread_id,
    }
}

_printed = set()

# Iterate over the tutorial questions and simulate the conversation with the agent
for question in tutorial_questions:
    # Stream the events from the graph, processing the user's question
    events = part_1_graph.stream(
        {"messages": ("user", question)}, config, stream_mode="values"
    )
    
    # Print the responses from the agent for each event
    for event in events:
        _print_event(event, _printed)


================================ Human Message =================================

Write a documentation for the method COPY_CLGRP_COND
Configuration received: {'metadata': {'thread_id': '853a9727-8010-44f5-835b-2a07696a1938', 'langgraph_step': 1, 'langgraph_node': 'assistant', 'langgraph_triggers': ['start:assistant'], 'langgraph_path': ('__pregel_pull', 'assistant'), 'langgraph_checkpoint_ns': 'assistant:8d6446ad-1dd4-d0b4-c5f8-9c3b3959cf5f'}, 'configurable': {'thread_id': '853a9727-8010-44f5-835b-2a07696a1938', '__pregel_resuming': False, '__pregel_task_id': '8d6446ad-1dd4-d0b4-c5f8-9c3b3959cf5f', '__pregel_send': functools.partial(<function local_write at 0x00000272EC388B80>, <built-in method extend of collections.deque object at 0x00000272F0354400>, dict_keys(['__start__', 'assistant', 'tools'])), '__pregel_read': functools.partial(<function local_read at 0x00000272EC388AF0>, 1, {'v': 1, 'ts': '2024-10-17T13:06:16.081036+00:00', 'id': '1ef8c889-7bab-617f-8000-5c477c16395f', 'channe